In [221]:
# Import standard libraries
import pandas as pd
import numpy as np

# Import libraries from surprise
from surprise import Dataset
from surprise import Reader
from surprise import SVD
from surprise import KNNWithMeans
from surprise import accuracy
from surprise.model_selection import train_test_split
from surprise.model_selection import KFold
from surprise.model_selection import GridSearchCV
from surprise.prediction_algorithms.predictions import Prediction

<span style="font-family: Arial; font-weight:bold;font-size:2em;color:#00b3e5;">  1. Import and explore data:

In [144]:
cols_list = ["phone_url","date","lang","country","source","domain","score","score_max","author","product"]

In [145]:
data_file_1 = pd.read_csv('phone_user_review_file_1.csv',encoding='latin1',usecols=cols_list)
data_file_1["extract"] = pd.read_csv('phone_user_review_file_1.csv',encoding='latin1',usecols=["extract"])
print(data_file_1.size)
print(data_file_1.shape)
print(data_file_1.info())

4124010
(374910, 11)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 374910 entries, 0 to 374909
Data columns (total 11 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   phone_url  374910 non-null  object 
 1   date       374910 non-null  object 
 2   lang       374910 non-null  object 
 3   country    374910 non-null  object 
 4   source     374910 non-null  object 
 5   domain     374910 non-null  object 
 6   score      366691 non-null  float64
 7   score_max  366691 non-null  float64
 8   author     371641 non-null  object 
 9   product    374910 non-null  object 
 10  extract    371934 non-null  object 
dtypes: float64(2), object(9)
memory usage: 31.5+ MB
None


In [146]:
data_file_1.dtypes

phone_url     object
date          object
lang          object
country       object
source        object
domain        object
score        float64
score_max    float64
author        object
product       object
extract       object
dtype: object

In [147]:
data_file_1.head()

,phone_url,date,lang,country,source,domain,score,score_max,author,product,extract
0,/cellphones/samsung-galaxy-s8/,5/2/2017,en,us,Verizon Wireless,verizonwireless.com,10.0,10.0,CarolAnn35,Samsung Galaxy S8,As a diehard Samsung fan who has had every Sam...
1,/cellphones/samsung-galaxy-s8/,4/28/2017,en,us,Phone Arena,phonearena.com,10.0,10.0,james0923,Samsung Galaxy S8,Love the phone. the phone is sleek and smooth ...
2,/cellphones/samsung-galaxy-s8/,5/4/2017,en,us,Amazon,amazon.com,6.0,10.0,R. Craig,"Samsung Galaxy S8 (64GB) G950U 5.8"" 4G LTE Unl...",Adequate feel. Nice heft. Processor's still sl...
3,/cellphones/samsung-galaxy-s8/,5/2/2017,en,us,Samsung,samsung.com,9.2,10.0,Buster2020,Samsung Galaxy S8 64GB (AT&T),Never disappointed. One of the reasons I've be...
4,/cellphones/samsung-galaxy-s8/,5/11/2017,en,us,Verizon Wireless,verizonwireless.com,4.0,10.0,S Ate Mine,Samsung Galaxy S8,I've now found that i'm in a group of people t...


In [148]:
data_file_2 = pd.read_csv('phone_user_review_file_2.csv',encoding='latin1',usecols=cols_list)
data_file_2["extract"] = pd.read_csv('phone_user_review_file_2.csv',encoding='latin1',usecols=["extract"])
print(data_file_2.size)
print(data_file_2.shape)
print(data_file_2.info())

1264175
(114925, 11)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114925 entries, 0 to 114924
Data columns (total 11 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   phone_url  114925 non-null  object 
 1   date       114925 non-null  object 
 2   lang       114925 non-null  object 
 3   country    114925 non-null  object 
 4   source     114925 non-null  object 
 5   domain     114925 non-null  object 
 6   score      112166 non-null  float64
 7   score_max  112166 non-null  float64
 8   author     113290 non-null  object 
 9   product    114925 non-null  object 
 10  extract    113965 non-null  object 
dtypes: float64(2), object(9)
memory usage: 9.6+ MB
None


In [149]:
data_file_2.dtypes

phone_url     object
date          object
lang          object
country       object
source        object
domain        object
score        float64
score_max    float64
author        object
product       object
extract       object
dtype: object

In [150]:
data_file_2.head()

,phone_url,date,lang,country,source,domain,score,score_max,author,product,extract
0,/cellphones/leagoo-lead-7/,4/15/2015,en,us,Amazon,amazon.com,2.0,10.0,luis,Leagoo Lead7 5.0 Inch HD JDI LTPS Screen 3G Sm...,"The telephone headset is of poor quality , not..."
1,/cellphones/leagoo-lead-7/,5/23/2015,en,gb,Amazon,amazon.co.uk,10.0,10.0,Mark Lavin,Leagoo Lead 7 Lead7 MTK6582 Quad core 1GB RAM ...,This is my first smartphone so I have nothing ...
2,/cellphones/leagoo-lead-7/,4/27/2015,en,gb,Amazon,amazon.co.uk,8.0,10.0,tracey,Leagoo Lead 7 Lead7 MTK6582 Quad core 1GB RAM ...,Great phone. Battery life not great but seems ...
3,/cellphones/leagoo-lead-7/,4/22/2015,en,gb,Amazon,amazon.co.uk,10.0,10.0,Reuben Ingram,Leagoo Lead 7 Lead7 MTK6582 Quad core 1GB RAM ...,Best 90 quid I've ever spent on a smart phone
4,/cellphones/leagoo-lead-7/,4/18/2015,en,gb,Amazon,amazon.co.uk,10.0,10.0,viorel,Leagoo Lead 7 Lead7 MTK6582 Quad core 1GB RAM ...,I m happy with this phone.it s very good.thx team


In [151]:
data_file_3 = pd.read_csv('phone_user_review_file_3.csv',encoding='latin1',usecols=cols_list)
data_file_3["extract"] = pd.read_csv('phone_user_review_file_3.csv',encoding='latin1',usecols=["extract"])
print(data_file_3.size)
print(data_file_3.shape)
print(data_file_3.info())

3442571
(312961, 11)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 312961 entries, 0 to 312960
Data columns (total 11 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   phone_url  312961 non-null  object 
 1   date       312961 non-null  object 
 2   lang       312961 non-null  object 
 3   country    312961 non-null  object 
 4   source     312961 non-null  object 
 5   domain     312961 non-null  object 
 6   score      304933 non-null  float64
 7   score_max  304933 non-null  float64
 8   author     302173 non-null  object 
 9   product    312960 non-null  object 
 10  extract    310231 non-null  object 
dtypes: float64(2), object(9)
memory usage: 26.3+ MB
None


In [152]:
data_file_3.dtypes

phone_url     object
date          object
lang          object
country       object
source        object
domain        object
score        float64
score_max    float64
author        object
product       object
extract       object
dtype: object

In [153]:
data_file_3.head()

,phone_url,date,lang,country,source,domain,score,score_max,author,product,extract
0,/cellphones/samsung-galaxy-s-iii-slim-sm-g3812/,11/7/2015,pt,br,Submarino,submarino.com.br,6.0,10.0,herlington tesch,Samsung Smartphone Samsung Galaxy S3 Slim G381...,"recomendo, eu comprei um, a um ano, e agora co..."
1,/cellphones/samsung-galaxy-s-iii-slim-sm-g3812/,10/2/2015,pt,br,Submarino,submarino.com.br,10.0,10.0,Luisa Silva Marieta,Samsung Smartphone Samsung Galaxy S3 Slim G381...,Comprei um pouco desconfiada do site e do celu...
2,/cellphones/samsung-galaxy-s-iii-slim-sm-g3812/,9/2/2015,pt,br,Submarino,submarino.com.br,10.0,10.0,Cyrus,Samsung Smartphone Samsung Galaxy S3 Slim G381...,"Muito bom o produto, obvio que tem versÃµes me..."
3,/cellphones/samsung-galaxy-s-iii-slim-sm-g3812/,9/2/2015,pt,br,Submarino,submarino.com.br,8.0,10.0,Marcela Santa Clara Brito,Samsung Smartphone Samsung Galaxy S3 Slim G381...,Unica ressalva fica para a camera que poderia ...
4,/cellphones/samsung-galaxy-s-iii-slim-sm-g3812/,9/1/2015,pt,br,Colombo,colombo.com.br,8.0,10.0,Claudine Maria Kuhn Walendorff,"Smartphone Samsung Galaxy S3 Slim, Dual Chip, ...",Rapidez e atenÃ§Ã£o na entrega. O aparelho Ã© ...


In [154]:
data_file_4 = pd.read_csv('phone_user_review_file_4.csv',encoding='latin1',usecols=cols_list)
data_file_4["extract"] = pd.read_csv('phone_user_review_file_4.csv',encoding='latin1',usecols=["extract"])
print(data_file_4.size)
print(data_file_4.shape)
print(data_file_4.info())

1081124
(98284, 11)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98284 entries, 0 to 98283
Data columns (total 11 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   phone_url  98284 non-null  object 
 1   date       98284 non-null  object 
 2   lang       98284 non-null  object 
 3   country    98284 non-null  object 
 4   source     98284 non-null  object 
 5   domain     98284 non-null  object 
 6   score      93706 non-null  float64
 7   score_max  93706 non-null  float64
 8   author     92696 non-null  object 
 9   product    98284 non-null  object 
 10  extract    96857 non-null  object 
dtypes: float64(2), object(9)
memory usage: 8.2+ MB
None


In [155]:
data_file_4.dtypes

phone_url     object
date          object
lang          object
country       object
source        object
domain        object
score        float64
score_max    float64
author        object
product       object
extract       object
dtype: object

In [156]:
data_file_4.head()

,phone_url,date,lang,country,source,domain,score,score_max,author,product,extract
0,/cellphones/samsung-s7262-duos-galaxy-ace/,3/11/2015,en,us,Amazon,amazon.com,2.0,10.0,Frances DeSimone,Samsung Galaxy Star Pro DUOS S7262 Unlocked Ce...,was not conpatable with my phone as stated. I ...
1,/cellphones/samsung-s7262-duos-galaxy-ace/,17/11/2015,en,in,Zopper,zopper.com,10.0,10.0,Expert Review,Samsung Galaxy Star Pro S7262 Black,Decent Functions and Easy to Operate Pros:- Th...
2,/cellphones/samsung-s7262-duos-galaxy-ace/,29/10/2015,en,in,Amazon,amazon.in,4.0,10.0,Amazon Customer,Samsung Galaxy Star Pro GT-S7262 (Midnight Black),Not Good Phone such price. Hang too much and v...
3,/cellphones/samsung-s7262-duos-galaxy-ace/,29/10/2015,en,in,Amazon,amazon.in,6.0,10.0,Amazon Customer,Samsung Galaxy Star Pro GT-S7262 (Midnight Black),not bad for features
4,/cellphones/samsung-s7262-duos-galaxy-ace/,29/10/2015,en,in,Amazon,amazon.in,10.0,10.0,NHK,Samsung Galaxy Star Pro GT-S7262 (Midnight Black),Excellent product


In [157]:
data_file_5 = pd.read_csv('phone_user_review_file_5.csv',encoding='latin1',usecols=cols_list)
data_file_5["extract"] = pd.read_csv('phone_user_review_file_5.csv',encoding='latin1',usecols=["extract"])
print(data_file_5.size)
print(data_file_5.shape)
print(data_file_5.info())

3852376
(350216, 11)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 350216 entries, 0 to 350215
Data columns (total 11 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   phone_url  350216 non-null  object 
 1   date       350216 non-null  object 
 2   lang       350216 non-null  object 
 3   country    350216 non-null  object 
 4   source     350216 non-null  object 
 5   domain     350216 non-null  object 
 6   score      321983 non-null  float64
 7   score_max  321983 non-null  float64
 8   author     321351 non-null  object 
 9   product    350216 non-null  object 
 10  extract    341836 non-null  object 
dtypes: float64(2), object(9)
memory usage: 29.4+ MB
None


In [158]:
data_file_5.dtypes

phone_url     object
date          object
lang          object
country       object
source        object
domain        object
score        float64
score_max    float64
author        object
product       object
extract       object
dtype: object

In [159]:
data_file_5.head()

,phone_url,date,lang,country,source,domain,score,score_max,author,product,extract
0,/cellphones/karbonn-k1616/,7/13/2016,en,in,91 Mobiles,91mobiles.com,2.0,10.0,venkatesh,Karbonn K1616,I bought 1 month before. currently speaker is ...
1,/cellphones/karbonn-k1616/,7/13/2016,en,in,91 Mobiles,91mobiles.com,6.0,10.0,Venkat,Karbonn K1616,"I just bought one week back, I have Airtel con..."
2,/cellphones/karbonn-k1616/,7/13/2016,en,in,91 Mobiles,91mobiles.com,4.0,10.0,krrish,Karbonn K1616,one problem in this handset opera is not worki...
3,/cellphones/karbonn-k1616/,4/25/2014,en,in,Naaptol,naaptol.com,10.0,10.0,BRIJESH CHAUHAN,Karbonn K1616 - Black,here Karbonn comes up with an another excellen...
4,/cellphones/karbonn-k1616/,4/23/2013,en,in,Naaptol,naaptol.com,10.0,10.0,Suraj CHAUHAN,Karbonn K1616 - Black,"What a phone, all so on Naaptol my god 23% off..."


In [160]:
data_file_6 = pd.read_csv('phone_user_review_file_6.csv',encoding='latin1',usecols=cols_list)
data_file_6["extract"] = pd.read_csv('phone_user_review_file_6.csv',encoding='latin1',usecols=["extract"])
print(data_file_6.size)
print(data_file_6.shape)
print(data_file_6.info())

1802207
(163837, 11)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 163837 entries, 0 to 163836
Data columns (total 11 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   phone_url  163837 non-null  object 
 1   date       163837 non-null  object 
 2   lang       163837 non-null  object 
 3   country    163837 non-null  object 
 4   source     163837 non-null  object 
 5   domain     163837 non-null  object 
 6   score      152165 non-null  float64
 7   score_max  152165 non-null  float64
 8   author     150780 non-null  object 
 9   product    163837 non-null  object 
 10  extract    160949 non-null  object 
dtypes: float64(2), object(9)
memory usage: 13.7+ MB
None


In [161]:
data_file_6.dtypes

phone_url     object
date          object
lang          object
country       object
source        object
domain        object
score        float64
score_max    float64
author        object
product       object
extract       object
dtype: object

In [162]:
data_file_6.head()

,phone_url,date,lang,country,source,domain,score,score_max,author,product,extract
0,/cellphones/samsung-instinct-sph-m800/,9/16/2011,en,us,Phone Arena,phonearena.com,8.0,10.0,ajabrams95,Samsung Instinct HD,I've had the phone for awhile and it's a prett...
1,/cellphones/samsung-instinct-sph-m800/,2/13/2014,en,us,Amazon,amazon.com,6.0,10.0,Stephanie,Samsung SPH M800 Instinct,to be clear it is not the sellers fault that t...
2,/cellphones/samsung-instinct-sph-m800/,12/30/2011,en,us,Phone Scoop,phonescoop.com,9.0,10.0,snickers,Instinct M800,Well i love this phone. i have had ton of phon...
3,/cellphones/samsung-instinct-sph-m800/,10/18/2008,en,us,HandCellPhone,handcellphone.com,4.0,10.0,A4C,Samsung Instinct,I have had my Instinct for several months now ...
4,/cellphones/samsung-instinct-sph-m800/,9/6/2008,en,us,Reviewed.com,reviewed.com,6.0,10.0,betaBgood,Samsung Instinct,i have had this instinct phone for about two m...


<span style="font-family: Arial; font-weight:bold;font-size:1.5em;color:#00b3e5;"> Merge the provided CSVs into one data-frame

In [163]:
data_raw = pd.concat([data_file_1,data_file_2,data_file_3,data_file_4,data_file_5,data_file_6])
data_raw=data_raw.reset_index(drop=True)

<span style="font-family: Arial; font-weight:bold;font-size:1.5em;color:#00b3e5;"> Check a few observations and shape of the data-frame

In [164]:
print(data_raw.size)
print(data_raw.shape)
print(data_raw.info())

15566463
(1415133, 11)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1415133 entries, 0 to 1415132
Data columns (total 11 columns):
 #   Column     Non-Null Count    Dtype  
---  ------     --------------    -----  
 0   phone_url  1415133 non-null  object 
 1   date       1415133 non-null  object 
 2   lang       1415133 non-null  object 
 3   country    1415133 non-null  object 
 4   source     1415133 non-null  object 
 5   domain     1415133 non-null  object 
 6   score      1351644 non-null  float64
 7   score_max  1351644 non-null  float64
 8   author     1351931 non-null  object 
 9   product    1415132 non-null  object 
 10  extract    1395772 non-null  object 
dtypes: float64(2), object(9)
memory usage: 118.8+ MB
None


In [165]:
data_raw.dtypes

phone_url     object
date          object
lang          object
country       object
source        object
domain        object
score        float64
score_max    float64
author        object
product       object
extract       object
dtype: object

In [166]:
data_raw.head()

,phone_url,date,lang,country,source,domain,score,score_max,author,product,extract
0,/cellphones/samsung-galaxy-s8/,5/2/2017,en,us,Verizon Wireless,verizonwireless.com,10.0,10.0,CarolAnn35,Samsung Galaxy S8,As a diehard Samsung fan who has had every Sam...
1,/cellphones/samsung-galaxy-s8/,4/28/2017,en,us,Phone Arena,phonearena.com,10.0,10.0,james0923,Samsung Galaxy S8,Love the phone. the phone is sleek and smooth ...
2,/cellphones/samsung-galaxy-s8/,5/4/2017,en,us,Amazon,amazon.com,6.0,10.0,R. Craig,"Samsung Galaxy S8 (64GB) G950U 5.8"" 4G LTE Unl...",Adequate feel. Nice heft. Processor's still sl...
3,/cellphones/samsung-galaxy-s8/,5/2/2017,en,us,Samsung,samsung.com,9.2,10.0,Buster2020,Samsung Galaxy S8 64GB (AT&T),Never disappointed. One of the reasons I've be...
4,/cellphones/samsung-galaxy-s8/,5/11/2017,en,us,Verizon Wireless,verizonwireless.com,4.0,10.0,S Ate Mine,Samsung Galaxy S8,I've now found that i'm in a group of people t...


In [167]:
data_raw.nunique()

phone_url       5556
date            7728
lang              22
country           42
source           331
domain           384
score             86
score_max          1
author        801103
product        61313
extract      1321353
dtype: int64

In [168]:
data_raw[data_raw.duplicated()==True]

,phone_url,date,lang,country,source,domain,score,score_max,author,product,extract
342,/cellphones/samsung-galaxy-s8/,4/26/2017,nl,nl,Coolblue,coolblue.nl,9.6,10.0,Stefanie Hinneman,Samsung Galaxy S8 Zwart,Ik vind het tot nu toe een heel erg aangenaam ...
343,/cellphones/samsung-galaxy-s8/,4/26/2017,nl,nl,Coolblue,coolblue.nl,9.6,10.0,Stefanie Hinneman,Samsung Galaxy S8 Zwart,Ik vind het tot nu toe een heel erg aangenaam ...
371,/cellphones/samsung-galaxy-s8/,4/25/2017,nl,nl,Coolblue,coolblue.nl,9.6,10.0,Robin Winckens,Samsung Galaxy S8 Zwart,Tot nu toe in de eerste dagen weinig tot geen ...
452,/cellphones/samsung-galaxy-s8/,4/25/2017,nl,nl,Coolblue,coolblue.nl,10.0,10.0,B Yapar,Samsung Galaxy S8 Zwart,Wat een huge scherm!
3718,/cellphones/samsung-galaxy-s6-edgeplus/,10/4/2015,en,us,Amazon,amazon.com,10.0,10.0,FrozenCloud,"Samsung Galaxy S6 Edge+, Black 64GB (Verizon W...","For readers, my opinions on this device (ATT 3..."
...,...,...,...,...,...,...,...,...,...,...,...
1398132,/cellphones/nokia-8850/,8/31/2010,ru,ru,Yandex,market.yandex.ru,10.0,10.0,NaN,Nokia 8850,"ÐÑÐ¾ÑÑÐ¾ Ð¾ÑÐ»Ð¸ÑÐ½ÑÐ¹ ÑÐµÐ»ÐµÑÐ¾Ð½, ..."
1398134,/cellphones/nokia-8850/,6/25/2010,ru,ru,Yandex,market.yandex.ru,10.0,10.0,NaN,Nokia 8850,"ÐÐ° ÑÐ²Ð¾Ð¸ Ð´ÐµÐ½ÑÐ³Ð¸, 1000 Ð´Ð¾Ð»Ð»Ð°ÑÐ..."
1410207,/cellphones/samsung-sgh-600/,4/30/2015,tr,tr,Cepworld,cepworld.com,NaN,NaN,muhsin,Samsung (936) SGH-600,varsa satan 05312614607
1411460,/cellphones/philips-genie-2000/,8/20/2000,it,it,Ciao,ciao.it,8.0,10.0,tonyblack,Philips Genie 2000,"Il Genie e' un buon telefono, scorrimente del ..."


In [169]:
data_raw.isnull().sum()

phone_url        0
date             0
lang             0
country          0
source           0
domain           0
score        63489
score_max    63489
author       63202
product          1
extract      19361
dtype: int64

<span style="font-family: Arial; font-weight:bold;font-size:2em;color:#00b3e5;">  2. Data cleansing

In [170]:
extract_data = data_raw['extract']
data = data_raw.drop(['extract'],axis=1)

<span style="font-family: Arial; font-weight:bold;font-size:1.5em;color:#00b3e5;"> Round off scores to the nearest integers

In [171]:
data.round({'score':1})

,phone_url,date,lang,country,source,domain,score,score_max,author,product
0,/cellphones/samsung-galaxy-s8/,5/2/2017,en,us,Verizon Wireless,verizonwireless.com,10.0,10.0,CarolAnn35,Samsung Galaxy S8
1,/cellphones/samsung-galaxy-s8/,4/28/2017,en,us,Phone Arena,phonearena.com,10.0,10.0,james0923,Samsung Galaxy S8
2,/cellphones/samsung-galaxy-s8/,5/4/2017,en,us,Amazon,amazon.com,6.0,10.0,R. Craig,"Samsung Galaxy S8 (64GB) G950U 5.8"" 4G LTE Unl..."
3,/cellphones/samsung-galaxy-s8/,5/2/2017,en,us,Samsung,samsung.com,9.2,10.0,Buster2020,Samsung Galaxy S8 64GB (AT&T)
4,/cellphones/samsung-galaxy-s8/,5/11/2017,en,us,Verizon Wireless,verizonwireless.com,4.0,10.0,S Ate Mine,Samsung Galaxy S8
...,...,...,...,...,...,...,...,...,...,...
1415128,/cellphones/alcatel-ot-club_1187/,5/12/2000,de,de,Ciao,ciao.de,2.0,10.0,david.paul,Alcatel Club Plus Handy
1415129,/cellphones/alcatel-ot-club_1187/,5/11/2000,de,de,Ciao,ciao.de,10.0,10.0,Christiane14,Alcatel Club Plus Handy
1415130,/cellphones/alcatel-ot-club_1187/,5/4/2000,de,de,Ciao,ciao.de,2.0,10.0,michaelawr,Alcatel Club Plus Handy
1415131,/cellphones/alcatel-ot-club_1187/,5/1/2000,de,de,Ciao,ciao.de,8.0,10.0,claudia0815,Alcatel Club Plus Handy


<span style="font-family: Arial; font-weight:bold;font-size:1.5em;color:#00b3e5;"> Check for duplicate values and remove them if there is any

In [172]:
data = data_raw.drop_duplicates(keep='first')

<span style="font-family: Arial; font-weight:bold;font-size:1.5em;color:#00b3e5;"> Check for missing values. Impute the missing values if there is any.

In [173]:
data.dropna(how = 'any', inplace = True)

C:\Users\kakar\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [174]:
print(data.size)
print(data.shape)

13985961
(1271451, 11)


In [175]:
data.isnull().sum()

phone_url    0
date         0
lang         0
country      0
source       0
domain       0
score        0
score_max    0
author       0
product      0
extract      0
dtype: int64

In [176]:
data.head()

,phone_url,date,lang,country,source,domain,score,score_max,author,product,extract
0,/cellphones/samsung-galaxy-s8/,5/2/2017,en,us,Verizon Wireless,verizonwireless.com,10.0,10.0,CarolAnn35,Samsung Galaxy S8,As a diehard Samsung fan who has had every Sam...
1,/cellphones/samsung-galaxy-s8/,4/28/2017,en,us,Phone Arena,phonearena.com,10.0,10.0,james0923,Samsung Galaxy S8,Love the phone. the phone is sleek and smooth ...
2,/cellphones/samsung-galaxy-s8/,5/4/2017,en,us,Amazon,amazon.com,6.0,10.0,R. Craig,"Samsung Galaxy S8 (64GB) G950U 5.8"" 4G LTE Unl...",Adequate feel. Nice heft. Processor's still sl...
3,/cellphones/samsung-galaxy-s8/,5/2/2017,en,us,Samsung,samsung.com,9.2,10.0,Buster2020,Samsung Galaxy S8 64GB (AT&T),Never disappointed. One of the reasons I've be...
4,/cellphones/samsung-galaxy-s8/,5/11/2017,en,us,Verizon Wireless,verizonwireless.com,4.0,10.0,S Ate Mine,Samsung Galaxy S8,I've now found that i'm in a group of people t...


<span style="font-family: Arial; font-weight:bold;font-size:1.5em;color:#00b3e5;"> Keep only 1000000 data samples. Use random state=612.

In [177]:
data = data.sample(1000000,random_state=612)
print(data.size)
print(data.shape)
data.head()

11000000
(1000000, 11)


,phone_url,date,lang,country,source,domain,score,score_max,author,product,extract
609806,/cellphones/nokia-lumia-1020/,11/20/2013,en,gb,Amazon,amazon.co.uk,10.0,10.0,jimperkins110,Lumia Nokia Lumia 1020 UK Sim Free Windows Sma...,"How did I live without it, excellent in every ..."
1372024,/cellphones/motorola-v557/,1/19/2006,en,us,Amazon,amazon.com,2.0,10.0,Penelope Brown,Motorola V557 Unlocked Quadband Cell Phone,I've had this phone for a little more than a y...
1110810,/cellphones/sanyo-scp-3810/,12/14/2009,en,us,Phone Scoop,phonescoop.com,4.0,10.0,JohnnyPasta,SCP-3810 / Mirro,I won't give it a rating so far. I just acquir...
532252,/cellphones/apple-iphone-5s/,8/26/2014,en,us,Amazon,amazon.com,10.0,10.0,Mitchell Coleman,Apple iPhone 5s 16GB 4G LTE GSM Gold - AT&T Wi...,"my in-laws really love the gifts iph5s, thank you"
1334423,/cellphones/motorola-l6/,1/28/2009,es,ec,MercadoLibre,opinion.mercadolibre.com.ec,6.0,10.0,JESSROSA,Motorola L6,ESTA BUENO BONITO Y BARATO


<span style="font-family: Arial; font-weight:bold;font-size:1.5em;color:#00b3e5;"> Drop irrelevant features. Keep features like Author, Product, and Score

In [178]:
data = data[['author','product','score']]
data.head()

,author,product,score
609806,jimperkins110,Lumia Nokia Lumia 1020 UK Sim Free Windows Sma...,10.0
1372024,Penelope Brown,Motorola V557 Unlocked Quadband Cell Phone,2.0
1110810,JohnnyPasta,SCP-3810 / Mirro,4.0
532252,Mitchell Coleman,Apple iPhone 5s 16GB 4G LTE GSM Gold - AT&T Wi...,10.0
1334423,JESSROSA,Motorola L6,6.0


In [179]:
data.nunique()

author     626247
product     51992
score          85
dtype: int64

<span style="font-family: Arial; font-weight:bold;font-size:2em;color:#00b3e5;">  3. Data analysis

<span style="font-family: Arial; font-weight:bold;font-size:1.5em;color:#00b3e5;"> Identify the most rated features

In [180]:
products_count = data.groupby(by=['product']).count()
score_column_products_count = products_count['score']
score_column_products_count.max()

4118

In [181]:
products_count['score'].idxmax()

'Lenovo Vibe K4 Note (White,16GB)'

<span style="font-family: Arial; font-weight:bold;font-size:1.5em;color:#00b3e5;"> Identify the users with most number of reviews


In [182]:
authors_count = data.groupby(by=['author']).count()
score_column_authors_count = authors_count['score']
score_column_authors_count.max()

60330

In [183]:
authors_count['score'].idxmax()

'Amazon Customer'

<span style="font-family: Arial; font-weight:bold;font-size:1.5em;color:#00b3e5;"> Select the data with products having more than 50 ratings and users who have given more than 50 ratings. 

In [184]:
repeated_products = products_count.loc[products_count['score']>50]

In [185]:
data_repeated_products = data[data['product'].isin(repeated_products.index)]
data_repeated_products

,author,product,score
1334423,JESSROSA,Motorola L6,6.0
1213429,Amazon Customer,Motorola Triumph Prepaid Android Phone (Virgin...,2.0
1087957,Gary D. Scott,Samsung SGH-A847 Rubgy 2 Rugged GSM Unlocked A...,6.0
230610,Lorise,"Microsoft Lumia 950 XL Smartphone, 5.7"", camer...",10.0
508581,Vishal P.,"Lenovo Vibe K5 (Gold, VoLTE update)",8.0
...,...,...,...
1052390,Chris,"Microsoft Nokia C1-01 Handy (4,6 cm (1,8 Zoll)...",8.0
419842,Etelop,BQ Aquaris E4.5 - Smartphone libre Android (pa...,10.0
1263546,DINAMICA99,Sony Ericsson W300,10.0
301402,Cliente Amazon,BQ Aquaris E5 HD - Smartphone libre Android (p...,2.0


In [186]:
repeated_authors = authors_count.loc[authors_count['score']>50]

In [187]:
data_repeated_authors = data[data['author'].isin(repeated_authors.index)]
data_repeated_authors

,author,product,score
1213429,Amazon Customer,Motorola Triumph Prepaid Android Phone (Virgin...,2.0
939258,Nico,HTC Flyer Tablet 16GB (17.8cm (7 Zoll) Touchcr...,10.0
467520,Amazon Customer,ZTE Blade S6 5'' IPS HD Screen Unlocked 4G Sma...,8.0
764130,alex,Huawei Y201 Pro - Smartphone libre (pantalla t...,2.0
478771,Amazon Customer,Sony Xperia XA Dual (Graphite Black),6.0
...,...,...,...
377195,Amazon Kunde,Microsoft Nokia Lumia 630 Single-SIM Smartphon...,10.0
149289,Amazon Customer,"Honor 6X (Grey, 32GB)",10.0
1052390,Chris,"Microsoft Nokia C1-01 Handy (4,6 cm (1,8 Zoll)...",8.0
301402,Cliente Amazon,BQ Aquaris E5 HD - Smartphone libre Android (p...,2.0


<span style="font-family: Arial; font-weight:bold;font-size:1.5em;color:#00b3e5;"> Report the shape of the final dataset.

In [188]:
data_final = data[(data['product'].isin(repeated_products.index)) & (data['author'].isin(repeated_authors.index))]
print(data_final.shape)
data_final

(111697, 3)


,author,product,score
1213429,Amazon Customer,Motorola Triumph Prepaid Android Phone (Virgin...,2.0
478771,Amazon Customer,Sony Xperia XA Dual (Graphite Black),6.0
74417,Amazon Customer,"OnePlus 3 (Soft Gold, 64 GB)",10.0
499887,Amazon Customer,"Lenovo PHAB Plus Tablet (6.8 inch, 32GB, Wi-Fi...",10.0
1306428,ÐÐ¸Ñ Ð°Ð¸Ð»,Nokia 2630,4.0
...,...,...,...
64934,Davide,"Huawei P9 Lite Smartphone, LTE, Display 5.2'' ...",10.0
149289,Amazon Customer,"Honor 6X (Grey, 32GB)",10.0
1052390,Chris,"Microsoft Nokia C1-01 Handy (4,6 cm (1,8 Zoll)...",8.0
301402,Cliente Amazon,BQ Aquaris E5 HD - Smartphone libre Android (p...,2.0


<span style="font-family: Arial; font-weight:bold;font-size:2em;color:#00b3e5;">  4. Building reccomender systems

<span style="font-family: Arial; font-weight:bold;font-size:1.5em;color:#00b3e5;"> Build a popularity based model and recommend top 5 mobile phones

In [189]:
data_final.groupby('product')['score'].mean().head()

product
3100 / 3120                                                                                                                                                                                     10.000000
5-Zoll- Android 4.2 Cubot P9 3G Smart Phone MTK6572 Dual Core 1.3GHz QHD IPS Schirm 512MB RAM 4GB ROM GPS 8MP...                                                                                10.000000
5.5-Inch Unlocked Lenovo A850 3G Smartphone-(960x540) Quad Core 4GB MT6582m 1331MHz Android 4.2 Dual Camera +Dual SIM -Black (Rooted + Google Play)                                              5.200000
AICEK Coque ASUS ZenFone 2 ZE550ML/ZE551ML, AICEK Etui Silicone Gel ASUS ZenFone 2 Housse Antichoc ZenFone 2 Transparente Souple Coque de Protection pour ASUS ZenFone 2(5.5 Pouces)             9.523810
AICEK Coque ASUS ZenFone 3 Max ZC520TL, AICEK Etui Silicone Gel ASUS ZenFone 3 Max Housse Antichoc ZenFone 3 Max Transparente Souple Coque de Protection pour ASUS ZenFone 3 Max(5.2 Pou

In [190]:
data_final.groupby('product')['score'].mean().sort_values(ascending=False).head()

product
3100 / 3120                       10.0
Storm 9530                        10.0
LG Optimus 4X HD P880             10.0
Nokia N81                         10.0
Nokia N8 Sim Free Mobile Phone    10.0
Name: score, dtype: float64

<span style="font-family: Arial; font-weight:bold;font-size:1.5em;color:#00b3e5;"> Build a collaborative filtering model using SVD

In [191]:
reader = Reader(rating_scale=(1, 10))
data_final_spformat = Dataset.load_from_df(data_final[['author', 'product', 'score']], reader)

In [192]:
train_set, test_set = train_test_split(data_final_spformat, test_size=.15)

In [193]:
SVD_algo = SVD()
SVD_algo.fit(train_set)

<span style="font-family: Arial; font-weight:bold;font-size:1.5em;color:#00b3e5;"> Predict score (average rating) for test users

In [194]:
SVD_predicted_ratings = SVD_algo.test(test_set)
SVD_predicted_ratings

[Prediction(uid='kevin', iid='ZTE Axon 7 Unlocked Smartphone,64GB Ion Gold (US Warranty)', r_ui=10.0, est=7.117381580149096, details={'was_impossible': False}),
 Prediction(uid='Cliente Amazon', iid='Asus ZenFone 2 Laser 5.5" Smartphone, 16 GB, Dual SIM, Nero [Italia]', r_ui=10.0, est=9.003480971229983, details={'was_impossible': False}),
 Prediction(uid='Carlos', iid='Sim Free Sony Xperia XA Mobile Phone - Black', r_ui=8.0, est=8.864364405285531, details={'was_impossible': False}),
 Prediction(uid='Amazon Customer', iid='OnePlus 3 (Graphite, 64 GB)', r_ui=2.0, est=8.473920663532166, details={'was_impossible': False}),
 Prediction(uid='e-bit', iid='Smartphone Samsung Galaxy S4 Mini GT-I9192', r_ui=10.0, est=9.006362059490245, details={'was_impossible': False}),
 Prediction(uid='Amazon Customer', iid='Lenovo Used Lenovo Zuk Z1 (Space Grey, 64GB)', r_ui=2.0, est=5.4689852290066, details={'was_impossible': False}),
 Prediction(uid='Amazon Customer', iid='OnePlus 3 (Graphite, 64 GB)', r_ui

<span style="font-family: Arial; font-weight:bold;font-size:1.5em;color:#00b3e5;"> Evaluate the collaborative model. Print RMSE value.

In [195]:
accuracy.rmse(SVD_predicted_ratings, verbose=True)

RMSE: 2.6988


2.698765393258862

<span style="font-family: Arial; font-weight:bold;font-size:1.5em;color:#00b3e5;"> Build a collaborative filtering model using KNNwithMeans - user based

In [225]:
KNN_algo_user_based = KNNWithMeans(k=50, sim_options={'name': 'pearson_baseline', 'user_based': True})
KNN_algo_user_based.fit(train_set)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


<span style="font-family: Arial; font-weight:bold;font-size:1.5em;color:#00b3e5;"> Predict score (average rating) for test users - user based

In [226]:
KNN_predicted_ratings_user_based = KNN_algo_user_based.test(test_set)
KNN_predicted_ratings_user_based

[Prediction(uid='Jose', iid='Alcatel One Touch Idol 3 - Terminal libre (Android L, pantalla de 4.7" HD, cÃ¡mara de 13 MP, 8 GB, Quad Core 1.2 GHz, 1.5 GB de RAM), Gris oscuro', r_ui=10.0, est=6.783134187465677, details={'actual_k': 5, 'was_impossible': False}),
 Prediction(uid='Amazon Customer', iid='YU Yuphoria YU5010A (Black+Silver)', r_ui=10.0, est=6.800000000000001, details={'actual_k': 50, 'was_impossible': False}),
 Prediction(uid='Cliente Amazon', iid='Doogee Valencia2 Y100 Pro - 4G LTE Smartphone Libre Android 5.1(Quad Core, 5.0" Pantalla HD IPS, Ram 2GB, 16GB Rom, CÃ¡mara 8MP, Gps) Celular - Plateado', r_ui=6.0, est=6.857142857142856, details={'actual_k': 21, 'was_impossible': False}),
 Prediction(uid='Amazon Customer', iid='Motorola Moto X (2nd Generation) - Black Leather - 16 GB (U.S. Warranty) Unlocked Phone', r_ui=8.0, est=6.864791579668232, details={'actual_k': 8, 'was_impossible': False}),
 Prediction(uid='Amazon Customer', iid='Lenovo Used Lenovo Zuk Z1 (Space Grey, 64G

<span style="font-family: Arial; font-weight:bold;font-size:1.5em;color:#00b3e5;"> Build a collaborative filtering model using KNNwithMeans - item based

In [212]:
KNN_algo_item_based = KNNWithMeans(k=50, sim_options={'name': 'pearson_baseline', 'user_based': False})
KNN_algo_item_based.fit(train_set)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


<span style="font-family: Arial; font-weight:bold;font-size:1.5em;color:#00b3e5;"> Predict score (average rating) for test users - item based

In [213]:
KNN_predicted_ratings_item_based = KNN_algo_item_based.test(test_set)
KNN_predicted_ratings_item_based

[Prediction(uid='Florian', iid='Lenovo Motorola Moto E 2. Generation Smartphone mit 4G LTE (11,4 cm (4,5 Zoll) qHD-Touchscreen-Display mit 540 x 960 Pixel, 5 Megapixel Kamera, 1,2GHz Quad-Core-Prozessor, 1GB RAM, 8GB interner Speicher, Micro-SIM, Android 5 Lollipop) weiÃ\x9f', r_ui=10.0, est=7.447136104852753, details={'actual_k': 10, 'was_impossible': False}),
 Prediction(uid='Andrea', iid='Huawei Ascend Mate 7 Smartphone (6 Zoll (15,2 cm) Touch-Display, 16 GB Speicher, Android 4.4) silber', r_ui=10.0, est=9.363567895282516, details={'actual_k': 50, 'was_impossible': False}),
 Prediction(uid='Amazon Customer', iid='Lenovo Vibe K5 (Grey, VoLTE update)', r_ui=8.0, est=5.8, details={'actual_k': 50, 'was_impossible': False}),
 Prediction(uid='Peter', iid='Huawei Honor 8 Unlocked Smartphone 32 GB Dual Camera - US Warranty (Midnight Black)', r_ui=2.0, est=8.62747489440061, details={'actual_k': 50, 'was_impossible': False}),
 Prediction(uid='Amazon Customer', iid='LG G3, Metallic Black 32GB 

<span style="font-family: Arial; font-weight:bold;font-size:1.5em;color:#00b3e5;"> Evaluate the collaborative model. Print RMSE value

In [216]:
accuracy.rmse(KNN_predicted_ratings_user_based, verbose=True)

RMSE: 2.7644


2.764398475313179

In [217]:
accuracy.rmse(KNN_predicted_ratings_item_based, verbose=True)

RMSE: 2.7089


2.708911127949515

<span style="font-family: Arial; font-weight:bold;font-size:1.5em;color:#00b3e5;"> Observations and inferences

- The attribute 'was_impossible' of the predictions is mostly set to 'False' while using both SVD and KNNwithMeans algorithms.
- This implies that algorithms are succesful in finding the similar users and items for all the users and items respectively.
- In the case of both user-based and item-based collborative filtering using KNNwithMeans, it is observed that higher values of K result in a more error between actual ratings and the predicted ratings.
- This observation can be explained by the fact that usually higher variance is considered while predicting the ratings of the items for the higher values of K.
- The RMSE value using SVD is obtained as 2.69 while using user-based collborative filtering, it is 2.76 and using item-based collborative filtering, it is 2.70.
- This suggests that there is no much difference in the error while using the algorithms considered here.

<span style="font-family: Arial; font-weight:bold;font-size:1.5em;color:#00b3e5;"> Top 5 products for test users

In [273]:
author_id_list = []
for i in range(0,len(test_set)-1):
    author_id_list.append(test_set[i][0])
author_id_list = np.array(author_id_list)
author_id_list = np.unique(author_id_list)

In [275]:
product_id_list = data_final['product'].tolist()

for author_id in author_id_list:
    print('Predictions for:',author_id)
    i = 0
    for product_id in product_id_list:
        ratings_prediction = KNN_algo_user_based.predict(uid=author_id,iid=product_id)
        if ratings_prediction.est == 10:
            print('                ',product_id)
            i+=1
            if i>5:
                break

Predictions for: #
                 Samsung C3350 Handy (5,6 cm (2,2 Zoll) Display, 2 Megapixel Kamera) steel-gray
                 Samsung G935 Galaxy S7 Edge Smartphone da 32GB, Argento [Italia]
                 BQ Aquaris E5 HD - Smartphone libre Android (pantalla 5 pulgadas, cÃ¡mara 13 Mp, 16 GB, Quad-Core 1.3 GHz, 1 GB RAM), blanco y negro
                 Huawei P8 Lite wit / 16 GB
                 Huawei P8 Lite wit / 16 GB
                 Meizu M2 Note Smartphone, 5.5" Full HD, 4G, 13MPX, Dual SIM, 16GB, Bianco [EU]
Predictions for: ????????
                 Samsung Galaxy S7 Edge 32GB UK SIM-Free Smartphone - Gold
                 Samsung C3350 Handy (5,6 cm (2,2 Zoll) Display, 2 Megapixel Kamera) steel-gray
                 Huawei Ascend Y550 Smartphone (11,4 cm (4,5 Zoll) IPS-Display, 5 Megapixel-Kamera, 4GB Interner Speicher, Android 4.4) schwarz
                 Lenovo Vibe K4 Note (White,16GB)
                 Samsung Galaxy S5 SM-G900A GSM Unlocked Cellphone, 16GB, Blac

<span style="font-family: Arial; font-weight:bold;font-size:1.5em;color:#00b3e5;"> Cross validation techniques to get better results

In [199]:
kf = KFold(n_splits=10)

for train_set, test_set in kf.split(data_final_spformat):

    # train and test algorithm.
    SVD_algo.fit(train_set)
    SVD_predicted_ratings = SVD_algo.test(test_set)

    # Compute and print Root Mean Squared Error
    accuracy.rmse(SVD_predicted_ratings, verbose=True)

RMSE: 2.7433
RMSE: 2.7501
RMSE: 2.7803
RMSE: 2.7581
RMSE: 2.7437
RMSE: 2.7492
RMSE: 2.7162
RMSE: 2.7632
RMSE: 2.7532
RMSE: 2.7009


In [200]:
param_grid = {'n_epochs': [5, 10], 'lr_all': [0.002, 0.005],
              'reg_all': [0.4, 0.6]}
gs = GridSearchCV(SVD, param_grid, measures=['rmse'], cv=10)

gs.fit(data_final_spformat)

# best RMSE score
print(gs.best_score['rmse'])

# combination of parameters that gave the best RMSE score
print(gs.best_params['rmse'])

2.658191043575014
{'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.4}


<span style="font-family: Arial; font-weight:bold;font-size:1.5em;color:#00b3e5;"> Business scenario for using popularity based Recommendation Systems

When launching an ecommerce website for the first, the user specific data is unavailable. In this scenario, the popularity based recommendation systems are useful for providing recommendations to the customers who are visting the website for the first time.

<span style="font-family: Arial; font-weight:bold;font-size:1.5em;color:#00b3e5;"> Business scenario for using collaborative filtering based Recommendation Systems

When the ecommerce website has acquired good number users and also has listed several items then the collaborative filtering based recommendation systems can be employed to provide recommendations to the customers who already made a purchase through the website. 

<span style="font-family: Arial; font-weight:bold;font-size:1.5em;color:#00b3e5;"> Suggestions for improvements

- The data seems to be biased towards certain languages/countries. 
- This may lead to a scenario in which an item maybe recommended to an user in a country where the item is not popular. 
- So, a uniformity with respect to the language/country of origin of the reviews has to be sought.
- There are several customers that have anonymous names. It would be difficult for the algorithms to distinguish between the users with the same user id. So, it is very important to give a unque id for each customer while collecting the data.